In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv("autism_screening.csv")

data['age'] = data['age'].fillna(round(data['age'].mean()))
data['ethnicity'] = data['ethnicity'].replace('?', 'others')

from sklearn.preprocessing import LabelEncoder

# Select the categorical columns for label encoding
categorical_cols = ['gender', 'ethnicity', 'jundice', 'austim', 'contry_of_res', 'used_app_before', 'relation', 'Class/ASD']

# Perform label encoding
encoder = LabelEncoder()
for col in categorical_cols:
    data[col] = encoder.fit_transform(data[col])

data = data.drop('age_desc', axis=1)

X = data.drop('Class/ASD', axis=1)
y = data['Class/ASD']   

# Splitting the dataset into train and test sets: 70-30 split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3, random_state = 12)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the input data for RNN
X_train_rnn = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_rnn = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define the model architecture
model_rnn = Sequential([
    SimpleRNN(64, input_shape=(X_train_rnn.shape[1], 1), activation='relu'),
    Dense(1, activation='sigmoid')
])

model_rnn.summary()

# Compile the model
model_rnn.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

# Train the model
history_rnn = model_rnn.fit(X_train_rnn, y_train, epochs=10, batch_size=32, validation_split=0.2)


# Evaluate the model
loss_rnn, accuracy_rnn = model_rnn.evaluate(X_test_rnn, y_test)
print(f'RNN: Loss: {loss_rnn}, Accuracy: {accuracy_rnn}')





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 64)                4224      
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 4289 (16.75 KB)
Trainable params: 4289 (16.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________

Epoch 1/10


13/13 [==============================] - 1s 21ms/step - loss: 0.6071 - accuracy: 0.8855 - val_loss: 0.5338 - val_accuracy: 0.9899
Epoch 2/10
13/13 [==============================] - 0s 5ms/step - loss: 0.4399 - accuracy: 0.9644 - val_loss: 0.3063 - val_accuracy: 0.8586
Epoch 3/10
13/13 [==============================] - 0s 5ms/step - loss: 0.2288 - accuracy: 0.9033 - val_loss: 0.1984 - val_accuracy:

In [2]:
# Save the model to a file
model_rnn.save('best_model.h5')

w:\MyLap\techno-lap\Contributions\autism-detection-backend\.venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
